<div>
<img src="figures/svtLogo.png"/>
</div>
<h1><center>Mathematical Optimization for Engineers</center></h1>
<h2><center>Lab 14 - Uncertainty</center></h2>

We want to optimize the total annualized cost of a heating and electric power system. Three different technologies are present: 
- a gas boiler
- a combined heat and power plant
- a photovoltaic module

We first the the nominal case without uncertanties. 
Next, we will consider a two-stage approach to consider uncertainties in the electricity demand and the power producable via PV. 
Uncertain variables are the solar power and the power demand. 

In [ ]:
# import cell
from scipy.optimize import minimize, NonlinearConstraint, Bounds

In [ ]:
class Boiler():
    """Boiler 
    Gas in, heat out
    """
    
    def __init__(self):
        self.M = 0.75  
        
    def invest_cost(self, Qdot_nom):
        inv = 100 * Qdot_nom ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas = 60
        cost_gas_oper = Qdot_nom * cost_gas * op_load
        
        return cost_gas_oper
    
    def heat(self, Qdot_nom, op_load):
        eta_th = 0.9 - (1 - op_load) * 0.05
        return Qdot_nom * op_load * eta_th
    

In [ ]:
class CHP():
    """Combined-heat-and-power (CHP) engine 
    Gas in, heat and power out
    """

    def __init__(self):
        self.c_ref = 150
        self.M = 0.80  # [-], cost exponent
        self.cost_gas = 60
    
    def invest_cost(self, Qdot_nom):
        inv = self.c_ref * (Qdot_nom) ** self.M
        return inv
    
    def oper_cost(self, Qdot_nom, op_load): 
        cost_gas_oper = Qdot_nom * op_load * self.cost_gas
        return cost_gas_oper
    
    def elec_out(self, Qdot_nom, op_load):
        eta_el = 0.3 - (1 - op_load) * 0.1
        out_pow = eta_el * Qdot_nom * op_load
        return out_pow
    
    def heat(self, Qdot_nom, op_load): 
        eta_th = 0.6 - (1-op_load) * 0.05  
        return Qdot_nom * eta_th * op_load


In [ ]:
class PV:
    """Photovoltaic modules (PV) 
    solar 
    """ 
    
    def __init__(self): 
        self.M = 0.9  # [-], cost exponent
       
    def invest_cost(self, p_nom):
        inv = 200 * p_nom ** self.M
        return inv
    
    def oper_cost(self, out_nom): 
        return 0
    
    def elec_out(self, p_nom, op_load, solar):
        return p_nom * op_load * solar
    

In [ ]:
def objective_function(x, PV, Boiler, CHP, scenarios):
    total_cost = 0
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 
    
    # investment costs
    PV_cost = PV.invest_cost(design_PV) 
    Boiler_cost = Boiler.invest_cost(design_boiler)  
    CHP_cost = CHP.invest_cost(design_CHP) 

    total_cost = PV_cost + Boiler_cost + CHP_cost
    
    # expected operating costs
    operating_cost = 0
    
    idx_shift = 3
    for idx, iSec in enumerate(scenarios): 
        indexOffset = 3 + idx * idx_shift
        
        op_cost = Boiler.oper_cost(design_boiler, x[indexOffset + 1]) \
             + CHP.oper_cost(design_CHP, x[indexOffset + 2])
        total_cost = total_cost + iSec["p"] * op_cost
   
    return total_cost

In [ ]:
def constraint_function(x, PV, Boiler, CHP, scenarios): 
    heat_demand = 200
    
    design_PV = x[0]  
    design_boiler = x[1]  
    design_CHP = x[2] 

    # loop over all uncertatintes
    idx_shift = 3
    for idx, iSec in enumerate(scenarios): 
        idx_offset = 3 + idx * idx_shift
        elec_demand = iSec["elec"]
        
        # heat demand
        c.append(Boiler.heat(design_boiler, x[idx_offset + 1]) \
             + CHP.heat(design_CHP, x[idx_offset + 2]) - heat_demand)     
        # electricty demand 
        c.append(PV.elec_out(design_PV, x[idx_offset], iSec["solar"])
              + CHP.elec_out(design_CHP, x[idx_offset + 2]) - elec_demand)
   
    return c

In [ ]:
def print_solution(x):
    print('PV design: ', x[0])
    print('Boiler design: ', x[1])
    print('CHP design: ', x[2])
    
    # print scenarios
    n_scenarios = int((len(x) - 3) / 3)
    for i_scenario in range(1, n_scenarios + 1): 
            print('Scenario ' + str(i_scenario) + ' PV load: ', x[3 * i_scenario])
            print('Scenario ' + str(i_scenario) + ' Boiler load: ', x[3 * i_scenario + 1])
            print('Scenario ' + str(i_scenario) + ' CHP load: ', x[3 * i_scenario + 2], end='\n\n')
    

In [ ]:
# nominal case
scenario1 = {"p": 1.0, "solar":1.0, "elec": 100}
scenarios = [scenario1] # base scenario


In [ ]:
# now consider different scenarios
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
# constraints need bounds
nlc = NonlinearConstraint(cons, 0, 0)
# bounds for operation 0 . 1
x_guess = [200,200,200, 1,1,1 ]
# bounds for decision variables
lbs = [0, 0, 0, 0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1]
bnds = Bounds(lbs, ubs)


In [ ]:
res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nlc,
               options={"maxiter": 25, 'iprint': 2, 'disp': True})


In [ ]:
print_solution(res.x)

In [ ]:
# nominal 
# uncertanties: power demand and solar power (relative 1.0)
scenario1 = {"p": 0.4, "solar":1.0, "elec": 100}
scenario2 = {"p": 0.3, "solar":1.0, "elec": 120}
scenario3 = {"p": 0.3, "solar":0.5, "elec": 80}

# put scenarios together
scenarios = [scenario1] # some base scenario
scenarios.append(scenario2)
scenarios.append(scenario3)

print(scenarios)

In [ ]:
myPV = PV()
myBoiler = Boiler()
myCHP = CHP()
cons = lambda x: constraint_function(x, myPV, myBoiler, myCHP, scenarios)
obj = lambda x: objective_function(x, myPV, myBoiler, myCHP, scenarios)
# bounds and constraints
nonlinear_constraints = NonlinearConstraint(cons, 0, 0)
x_guess = [200,200,200, 1,1,1, 1,1,1, 1,1,1 ]
lbs = [0, 0, 0,   0, 0, 0,  0, 0, 0,  0, 0, 0]
ubs = [10000, 10000, 10000, 1, 1, 1, 1, 1, 1, 1, 1, 1]
bnds = Bounds(lbs, ubs)


res = minimize(obj, x_guess, method = 'SLSQP', bounds=bnds,
               constraints = nonlinear_constraints,
               options={"maxiter": 15, 'iprint': 2, 'disp': True})

In [ ]:
print_solution(res.x)